# Ноутбук для анализа исторических котировок акций для статьи на habr.com

## Данные по акциям российских эммитентов
Данные необходимо получить вручную, скачав в сайта https://www.finam.ru/profile/moex-akcii/gazprom/export/ и выбрав:
- саму акцию
- максимально возможный интервал для акции
- формат записи в файл - TICKER, PER, DATE, TIME, CLOSE
- добавить заголовок файла = Нет

Файл данных будет в таком виде:
```
GAZP,D,20060123,000000,218.8900000
GAZP,D,20060124,000000,224.0000000
GAZP,D,20060125,000000,228.3800000
GAZP,D,20060126,000000,224.4700000
GAZP,D,20060127,000000,228.7500000
```

Анализироваться будут ВСЕ акции, файлы по которым будут подложены а папку ./data

## Данные по иностранным акциям
Тикеры для иностранных акций искать на finance.yahoo.com (например https://finance.yahoo.com/quote/ZG) и затем нужный тикер добавить в список ниже по тексту ноутбука

In [1]:
import os
import yfinance as yf
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

In [2]:
def get_stocks_from_csv(path: str) -> pd.DataFrame:
    """
    Загрузка данных из CSV-файлов для российских акций (файлы с Финама)
    """
    files = list(filter(lambda file_name: file_name[-4:] == '.txt', os.listdir(path)))
    columns = ['TICKER', 'PER', 'DATE', 'TIME', 'CLOSE']
    df = pd.DataFrame()
    for file in files:
        df = df.append(pd.read_csv(path + file, 
                                   header=None, 
                                   names = columns)
                      )
    df['DATE'] = pd.to_datetime(df['DATE'], format='%Y%m%d')
    return pd.DataFrame(df, columns = ['TICKER', 'DATE', 'CLOSE'])


def get_stocks_from_yahoo(tickers: list) -> pd.DataFrame:
    """
    Загрузка данных из API Yahoo Finance
    """
    df = pd.DataFrame()
    columns = ['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME', 'DIVIDENDS', 'STOCK_SPLITS', 'TICKER']
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="max")
        hist['TICKER'] = ticker      
        df = df.append(hist.rename_axis('DATA').reset_index())
    df.columns = columns
    return pd.DataFrame(df, columns = ['TICKER', 'DATE', 'CLOSE'])

In [3]:
def get_price(df, date):
    """
    Получение цены, если данные есть за каждый день
    Принимает на вход уже отфильтрованный по тикеру ДатаФрейм c данными
    """
    return df[df['DATE'] == date]['CLOSE'].head(1).iloc[0]

def get_empty_price(df, date):
    """
    Получение цены с учетом пропусков в выходные и праздничные дни
    Принимает на вход уже отфильтрованный по тикеру ДатаФрейм c данными
    """
    return df[df['DATE'] <= date].sort_values(by=['DATE'], ascending=False)['CLOSE'].head(1).iloc[0]

In [4]:
def fill_empty_dates(df, ticker):
    """
    Заполнение пропусков котировок в выходные и праздничные дни.
    Принимает на вход уже отфильтрованный по тикеру ДатаФрейм c данными
    """
    min_date = df['DATE'].min()
    max_date = df['DATE'].max() 
    for date in pd.date_range(min_date, max_date):
        if df[df['DATE'] == date].empty:
            new_row = {'TICKER': ticker, 'DATE': date, 'CLOSE': get_empty_price(df, date)}
            df = df.append(new_row, ignore_index=True)
    return df

In [5]:
def calc(df, ticker, term_years, barrier, coupon_term_month, barrier_reduce=0):
    """
    Основной расчет
    params: df - датафрейм с отфильтрованными по тикеру данными,
    params: ticker - тикер
    params: term_years - срок продукта в годах
    params: barrier - барьер поставки, в долях
    params: coupon_term_month - купонный период в месяцах
    """
    min_date = df['DATE'].min()
    max_date = df['DATE'].max() - relativedelta(years=term_years)
    if min_date >= max_date:
        print(f'По акции {ticker} недостаточно данных для анализа')
        return None
    total_cnt = 0       # всего дней
    barrier_cnt = 0     # дней с пробитием барьера
    autocall_cnt = 0    # дней с автоколлом
    autocoll_sum = 0    # для подсчета среднего срока до автоколла
    for t0 in pd.date_range(min_date, max_date):
        total_cnt += 1
        # Пробитие барьера
        price0 = get_price(df, t0)
        t = t0 + relativedelta(years=term_years)
        price = get_price(df, t)
        if price <= barrier*price0:
            barrier_cnt += 1
        # Автоколл
        for period_num in range(1, int(term_years*12/coupon_term_month)+1):
            t = t0 + relativedelta(months=period_num*coupon_term_month)
            price = get_price(df, t)
            if price > price0*(1 - barrier_reduce*period_num):
                autocall_cnt += 1
                autocoll_sum += period_num
                break
    return {'TICKER': ticker, 
            'MIN_DATE': min_date,
            'MAX_DATE': max_date, 
            'TOTAL_YEARS': round(total_cnt/365, 2),
            'BARRIER_DAYS': barrier_cnt,
            'BARRIER_PRC': round(barrier_cnt/total_cnt * 100, 2),
            'AUTOCALL_DAYS': autocall_cnt,
            'AUTOCALL_PRC': round(autocall_cnt/total_cnt * 100, 2),
            'AVG_AUTOCALL_PERIOD': round(autocoll_sum/autocall_cnt, 2)
           }     

### Основной расчет

In [7]:
# Источники данных
path = './data/' # Длф РФ


# Параметры структурной ноты 

# Акции РФ-1
#term_years = 1
#barrier = 0.85
#barrier_reduce = 0
#coupon_term_month = 1
#russian_stocks = True 

# Нота с иностранными акциями
term_years = 3
barrier = 1
barrier_reduce = 0.02
coupon_term_month = 3
russian_stocks = False 
US_Stocks = ['AMD', 'ATVI', 'EA', 'TTWO', 'UBI']


# Запускаем все
if russian_stocks:
    files = list(filter(lambda file_name: file_name[-4:] == '.txt', os.listdir(path)))
    data = get_stocks_from_csv(path) 
else:
    data = get_stocks_from_yahoo(US_Stocks)
tickers = list(data.TICKER.unique())
result = list()
for ticker in tickers:
    df = fill_empty_dates(data[data['TICKER'] == ticker], ticker)
    df_tmp = calc(df, ticker, term_years, barrier, coupon_term_month)
    if df_tmp:
        result.append(df_tmp)

print('Все просчитано!')

Все просчитано!


In [8]:
# Выведем результаты в более читаемом табличном виде
print('ПАРАМЕТРЫ РАСЧЕТА:')
print(f'Срок продукта, год: {term_years}')
print(f'Барьер поставки: {barrier*100}%')
print(f'Понижение барьера поставки, в купонный период: {barrier_reduce*100}%')
print(f'Купонный период, месяцев: {coupon_term_month}')
print('----------------')

# Вероятность срабатывания барьера
p = 1
for stock in result:
    p = p * (1 - stock['BARRIER_PRC']/100)
print(f'Оценка вероятности срабатывания барьера поставки = {round((1 - p)*100, 2)}%')

# Вероятность автоколла
for stock in result:
    p = p * stock['AUTOCALL_PRC']/100
print(f'Оценка вероятности автоколла по портфелю = {round(p*100, 2)}%')
print('----------------')

out = pd.DataFrame(result)
out[['TICKER', 'TOTAL_YEARS', 'BARRIER_PRC', 'AUTOCALL_PRC', 'AVG_AUTOCALL_PERIOD']]

ПАРАМЕТРЫ РАСЧЕТА:
Срок продукта, год: 3
Барьер поставки: 100%
Понижение барьера поставки, в купонный период: 2.0%
Купонный период, месяцев: 3
----------------
Оценка вероятности срабатывания барьера поставки = 100.0%
Оценка вероятности автоколла по портфелю = 0.0%
----------------


,TICKER,TOTAL_YEARS,BARRIER_PRC,AUTOCALL_PRC,AVG_AUTOCALL_PERIOD
0,AMD,38.40,43.82,86.36,2.35
1,ATVI,24.78,10.53,97.34,2.02
2,EA,28.88,20.94,94.59,2.00
3,TTWO,21.31,25.49,96.54,2.02
4,UBI,0.15,100.00,100.00,3.15
